In [4]:
input_files = [
  'edgar_allen_poe.txt',
  'robert_frost.txt',
]

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [6]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allen_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [7]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [8]:
len(Ytrain), len(Ytest)

(1618, 540)

In [9]:
train_text[:5]

['she rolls through an ether of sighs ',
 'nearest resembles worship oh remember',
 'are broken at a blow',
 'in heaven a spirit doth dwell',
 'you said youd seen the stone baptismal font']

In [10]:
Ytrain[:5]

[0, 0, 0, 0, 1]

In [11]:
idx = 1
word2idx = {'<unk>': 0}

In [12]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [13]:
word2idx

{'<unk>': 0,
 'she': 1,
 'rolls': 2,
 'through': 3,
 'an': 4,
 'ether': 5,
 'of': 6,
 'sighs': 7,
 'nearest': 8,
 'resembles': 9,
 'worship': 10,
 'oh': 11,
 'remember': 12,
 'are': 13,
 'broken': 14,
 'at': 15,
 'a': 16,
 'blow': 17,
 'in': 18,
 'heaven': 19,
 'spirit': 20,
 'doth': 21,
 'dwell': 22,
 'you': 23,
 'said': 24,
 'youd': 25,
 'seen': 26,
 'the': 27,
 'stone': 28,
 'baptismal': 29,
 'font': 30,
 'where': 31,
 'no': 32,
 'wind': 33,
 'dared': 34,
 'to': 35,
 'stir': 36,
 'unless': 37,
 'on': 38,
 'tiptoe': 39,
 'was': 40,
 'fair': 41,
 'palace': 42,
 'door': 43,
 'then': 44,
 'took': 45,
 'other': 46,
 'as': 47,
 'just': 48,
 'who': 49,
 'may': 50,
 'be': 51,
 'ahead': 52,
 'foot': 53,
 'his': 54,
 'small': 55,
 'fist': 56,
 'buried': 57,
 'bushy': 58,
 'hide': 59,
 'i': 60,
 'doubted': 61,
 'if': 62,
 'should': 63,
 'ever': 64,
 'come': 65,
 'back': 66,
 'because': 67,
 'it': 68,
 'grassy': 69,
 'and': 70,
 'wanted': 71,
 'wear': 72,
 'steeple': 73,
 'from': 74,
 'their': 

In [14]:
len(word2idx)

2518

In [15]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [16]:
train_text_int[100:105]

[[16, 381, 382, 312, 383],
 [27, 384, 271, 295, 6, 385, 386, 70, 387],
 [388, 389, 23, 390, 183, 64, 391, 392],
 [70, 16, 393, 394],
 [251, 395, 189, 396, 38, 397, 398, 183, 399]]

In [17]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [18]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [19]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [20]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [21]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33868974042027195, 0.6613102595797281)

In [22]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [23]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [24]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9987639060568603


In [25]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8388888888888889


In [26]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [27]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 546,    2],
       [   0, 1070]], dtype=int64)

In [28]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 99,  75],
       [ 12, 354]], dtype=int64)

In [29]:
f1_score(Ytrain, Ptrain)

0.9990662931839402

In [30]:
f1_score(Ytest, Ptest)

0.890566037735849